# Outline

- Who needs a priority queue? Emergency departments, data compression algorithms, etc.
- How they work? By organizing data so that the highest (or lowest) valued element is the next one to return.

## Example

Waiting room of an ER, in order of arrivals:

`[broken angle, stroke, sprained wrist, kidney stone]`

Reorganized as a priority queue:

`[stroke, broken angle, sprained wrist, kidney stone]`

After the first case (`stroke`) is taken, the list will be reorganized with the most severe case moved to the front.

`[kidney stone, broken angle, sprained wrist]`

Notice that the list is _not_ sorted; just the most important item, based on some metric, is placed at the front. The only requirement here is that `list[0]` is always the most important item. The remaining elements can be randomly organized as far as we are concerned.


## Naive priority queue

- Linear scan to find the most importnat element and bring it to the front.
- Implement with integers before writing a generic class.
- Intro to generics


In [1]:
class SimplePriorityQ:

    _DEFAULT_CAPACITY: int = 10

    def __init__(self, capacity: int = _DEFAULT_CAPACITY) -> None:
        """Initialize an empty priority queue with given capacity."""
        self._capacity: int = capacity
        # Initialize a list at the specified capacity. This way we
        # can place values directly to the list instead of appending,
        # treating it like an actual array.
        self._underlying: list[int] = [None] * self._capacity
        # Tracks how many elements are in the "array". The invariant
        # here is that 0 ≤ size ≤ capacity.
        self._size: int = 0

    def insert(self, value: int) -> None:
        """Insert value into the priority queue."""
        if self._size >= self._capacity:
            raise Exception("Priority queue is full")
        self._underlying[self._size] = value
        self._size += 1
        self._move_important_to_front()

    def _move_important_to_front(self):
        # Where is the most important element?
        max_idx = self._most_important_idx()
        # Swap positions, bringign the most imporant element to the front
        self._swap(0, max_idx)

    def _most_important_idx(self):
        """Finds and returns the position of the most important
        element in the underlying array"""
        max_idx = 0
        for i in range(1, self._size):
            if self._underlying[i] > self._underlying[max_idx]:
                max_idx = i
        return max_idx

    def _swap(self, position, with_position):
        """Swaps positions between two elements in the list. The code uses
        the basic three-variable trick instead of the Pythonic trick
        a,b = b,a
        for better illustration and portability in other languages.
        """
        temp = self._underlying[position]
        self._underlying[position] = self._underlying[with_position]
        self._underlying[with_position] = temp

    def remove_max(self) -> int:
        """Remove and return the maximum element from the priority queue."""
        if self.is_empty():
            raise Exception("Priority queue is empty")
        # Grab the first element, so that we can return it
        most_important = self._underlying[0]
        # Use a temporary array that is just 1 element shorter to
        # copy everything underlying array element except for the
        # first one.
        temp_list = [None] * (len(self._underlying) - 1)
        for i in range(1, (self._size)):
            temp_list[i - 1] = self._underlying[i]
        # Replace the underlying array with the shorter temp array
        self._underlying = temp_list
        # Adjust the size
        self._size -= 1
        # Make sure that the most important element in this shorter
        # underlynig array is now at its front
        self._move_important_to_front()
        # Done; return the removed element to the user.
        return most_important

    def is_empty(self) -> bool:
        """Return True if the priority queue is empty, False otherwise."""
        return (self._size) == 0

    def __bool__(self) -> bool:
        return self.is_empty()

    # String representation constants

    _Q_EMPTY = "Queue is empty."
    _Q_IN_QUEUE = f"in queue: ["
    _Q_CONTENTS_CLOSING = f" ]"
    _Q_CONTENTS_DELIMITER = f", "
    _Q_SINGULAR = f"element"
    _Q_PLURAL = f"{_Q_SINGULAR}s"

    def __str__(self) -> str:
        # Initialize return string to assume empty queue
        text = f"{self._Q_EMPTY}"
        # If queue is not empty prepare to populate return string with data
        if self._size > 0:
            # Prepare singular or plural noun, 1 element, 2 elements etc
            elements = self._Q_SINGULAR if self._size == 1 else self._Q_PLURAL
            text = f"{self._size} {elements} {self._Q_IN_QUEUE} {self._underlying[0]}"
            for i in range(1, self._size):
                text += f"{self._Q_CONTENTS_DELIMITER}{self._underlying[i]}"
            text += f"{self._Q_CONTENTS_CLOSING}"
        return text

The code above can be tested, quickly, with the following.

```python
q = SimplePriorityQ()
q.insert(1); q.insert(3); q.insert(0); q.insert(4)
print(q); print(q.remove_max()) # expect [ 4, 1, 0, 3 ],  4
print(q); print(q.remove_max()) # expect [ 3, 0, 1 ],     3
print(q); print(q.remove_max()) # expext [ 1, 0 ],        1
print(q); print(q.remove_max()) # expect [ 0 ],           0
print(q); print(q.remove_max()) # expect `Queue is empty.` then exception
```


The naive priority queue runs in $\mathcal O(n)$ time. This is not bad, but it can be done faster.

Consider the data structure below.

![](MaxHeapTree.jpg)

 The data in it seem all over the place but there are two notable properties. The largest value node is at the top of the structure. And each node its greater than or equal to both its immediate two nodes below it.  With reference to the nodes below, this arrangement can be formalized as $p \geq\max\{c_L, c_R\}$ where a parent node $p$ has a left and a right child, $c_L$ and $c_R$ respectively.

![](parent_child.jpg)